# Überblick 

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras_tuner as kt

from numpy import array
from numpy import hstack
from numpy import empty
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from os import chdir
from os import getcwd
from os import listdir
import math

csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Sweden_2'
chdir(csv_file_path)

Y_all = pd.read_csv('heads.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)
X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)

Y = Y_all['2001-02-06':' 2015-12-29']#2001-01-01 to 2015-12-31
X = X_all['2001-01-07':' 2015-12-29']
# for sweden weekly forecasts are asked
#print(X.index, Y.index)

DatetimeIndex(['2001-01-07', '2001-01-08', '2001-01-09', '2001-01-10',
               '2001-01-11', '2001-01-12', '2001-01-13', '2001-01-14',
               '2001-01-15', '2001-01-16',
               ...
               '2015-12-20', '2015-12-21', '2015-12-22', '2015-12-23',
               '2015-12-24', '2015-12-25', '2015-12-26', '2015-12-27',
               '2015-12-28', '2015-12-29'],
              dtype='datetime64[ns]', name='time', length=5470, freq=None) DatetimeIndex(['2001-02-06', '2001-02-13', '2001-02-20', '2001-02-27',
               '2001-03-06', '2001-03-13', '2001-03-20', '2001-03-27',
               '2001-04-03', '2001-04-10',
               ...
               '2015-10-27', '2015-11-03', '2015-11-10', '2015-11-17',
               '2015-11-24', '2015-12-01', '2015-12-08', '2015-12-15',
               '2015-12-22', '2015-12-29'],
              dtype='datetime64[ns]', name='Date', length=778, freq=None)


In [2]:
X_train = X[:'2011-12-31']
X_test = X['2014-01-01':]
X_valid = X['2012-01-01':'2013-12-31']

Y_train = Y[:'2011-12-31']
Y_test = Y['2014-01-01':]
Y_valid = Y['2012-01-01':'2013-12-31']


csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\data'
chdir(csv_file_path)
from helper import *

(x_tr,y, x_te, y_test, x_va, y_valid, scaler_X,scaler_Y)=prepare_data_weekly(X_train,Y_train,X_valid,Y_valid,X_test,Y_test)
print(x_te.shape)
print(x_tr.shape)
print(x_va.shape)

iters=3
n_steps_in=30
n_steps_out=1

x_final_unsplit = scaler_X.transform(X)
y_final = scaler_Y.transform(Y)

x_final = split_sequences_x(x_final_unsplit, n_steps_in, n_steps_out)
print(x_final.shape, y_final.shape) 

(569, 1) y_train.shape transformed
(728, 9)
(4011, 9)
(731, 9)
(778, 30, 9) (778, 1)


777.1428571428571

In [3]:
import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

In [4]:
import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K

class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training
        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs


In [5]:
test_runs = 100
    
#load models 
csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge'
chdir(csv_file_path)

transformer_model = keras.models.load_model('TF_Sweden2_tune.h5')
transformer_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 9)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 30, 9)        18          input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 30, 9)        2427        layer_normalization[0][0]        
                                                                 layer_normalization[0][0]        
__________________________________________________________________________________________________
dropout (Dropout)               (None, 30, 9)        0           multi_head_attention[0][0]   

## Prediction Transformer

In [7]:

print(x_final.shape)
Y_pred_test_tr = np.empty((x_final.shape[0],n_steps_out,test_runs))
y_correct_utf = scaler_Y.inverse_transform(y_final)#test
for j in range(test_runs): 
    Y_pred_test_tr[:,:,j]=transformer_model.predict(x_final)
    yhat_utf = scaler_Y.inverse_transform(Y_pred_test_tr[:,:,j])

(778, 30, 9)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


## POC

In [19]:
print(Y_pred_test_tr.shape)
Y_pred = np.mean(Y_pred_test_tr, axis=2)
Y_std = np.std(Y_pred_test_tr, axis=2)

zT = 8.6
T_ival = [Y_pred - zT*Y_std,Y_pred + zT*Y_std]
print(T_ival[0].shape, y_final.shape)
#print(range(len(Y_test_day1)))
def calc_POC(list_of_bounds, measured):
    temp = 0 
    for i in range(len(measured)):
        if measured[i]< list_of_bounds[1][i] and measured[i]> list_of_bounds[0][i]:
            temp = temp + 1
    POC = temp/len(measured)
    return POC

print(T_ival[1].shape)
POC_tr = calc_POC(T_ival,y_final)
print(POC_tr)


(778, 1) (778, 1)
(778, 1)
0.9511568123393316


In [20]:
print(Y_pred_test_tr.shape)
Y_pred_test_tr_flat = Y_pred_test_tr.reshape((test_runs*Y_pred_test_tr.shape[0], 1))
Y_pred_test_utf_flat = scaler_Y.inverse_transform(Y_pred_test_tr_flat)
Y_pred_test_utf = Y_pred_test_utf_flat.reshape((Y_pred_test_tr.shape[0], 1,test_runs))
Y_pred_calib = np.mean(Y_pred_test_utf, axis=2)
Y_std_calib = np.std(Y_pred_test_utf, axis=2)
T_ival_calib = [Y_pred_calib - zT*Y_std_calib,Y_pred_calib + zT*Y_std_calib]
#T_ival_calib[1]

(778, 1, 100)


array([[348.97833742],
       [348.37960196],
       [348.48275259],
       [348.36450236],
       [348.55260202],
       [348.30334167],
       [347.6379814 ],
       [347.90991403],
       [347.97695377],
       [347.26495077],
       [347.74287905],
       [347.82961218],
       [348.45568282],
       [348.28854176],
       [348.80899748],
       [348.99670804],
       [349.03937513],
       [348.78535513],
       [348.96924177],
       [348.9936537 ],
       [349.28651691],
       [349.23697772],
       [349.36347877],
       [349.27756147],
       [349.63279585],
       [349.39912555],
       [349.57068842],
       [349.37587002],
       [349.321523  ],
       [349.28359548],
       [349.06341235],
       [349.46813748],
       [349.30108946],
       [349.08561195],
       [349.12139694],
       [349.12806172],
       [348.80455519],
       [348.78956422],
       [348.57111342],
       [348.83396842],
       [348.56232743],
       [348.75212913],
       [348.65091272],
       [348

In [21]:
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Sweden_2'
chdir(csv_file_path)

X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)

X_finaltest = X_all['2016-01-01':]
X_finaltest

,rr,tg,tn,tx,pp,hu,fg,qq,et
time,,,,,,,,,
2016-01-01,0.0,0.080000,-4.800000,2.530000,1023.3,74.235000,2.44,1.0,0.008901
2016-01-02,0.0,-4.810000,-10.870000,0.840000,1035.8,84.575005,1.33,1.0,0.007150
2016-01-03,0.0,-13.820000,-16.779999,-5.110000,1029.0,80.010000,0.88,1.0,0.004458
2016-01-04,0.0,-16.410000,-19.289999,-8.820000,1024.8,68.050000,1.53,1.0,0.003818
2016-01-05,0.0,-19.260000,-23.580000,-15.509999,1019.3,71.835000,1.01,1.0,0.003188
...,...,...,...,...,...,...,...,...,...
2021-12-27,0.0,-8.620000,-13.530000,-5.350000,1004.8,70.915000,2.45,0.0,0.000000
2021-12-28,0.0,-15.700000,-17.900000,-9.040000,1004.2,81.530000,1.94,0.0,0.000000
2021-12-29,0.0,-19.990000,-24.189999,-14.120000,1000.4,79.715004,2.25,0.0,0.000000


In [22]:
X_finaltest_trf = scaler_X.transform(X_finaltest)
X_ftest = split_sequences_x(X_finaltest_trf,n_steps_in, n_steps_out)
X_ftest.shape

(309, 30, 9)

In [23]:
print(x_final.shape)
Y_pred_ftest_tr = np.empty((X_ftest.shape[0],n_steps_out,test_runs))
for j in range(test_runs): 
    Y_pred_ftest_tr[:,:,j]=transformer_model.predict(X_ftest)

(778, 30, 9)


In [24]:

Y_pred_ftest_tr_flat = Y_pred_ftest_tr.reshape((100*Y_pred_ftest_tr.shape[0], 1))
Y_pred_ftest_utf_flat = scaler_Y.inverse_transform(Y_pred_ftest_tr_flat)
Y_pred_ftest_utf = Y_pred_ftest_utf_flat.reshape((Y_pred_ftest_tr.shape[0], 1,100))

yhat_mean = np.mean(Y_pred_ftest_utf, axis=2)
yhat_std = np.std(Y_pred_ftest_utf, axis=2)
T_ival_utf = [yhat_mean - zT*yhat_std,yhat_mean + zT*yhat_std]
yhat_mean.shape

In [26]:
final_idx_calib = pd.date_range(start='2001-02-06', end='2015-12-29', freq='7d')
print(final_idx_calib)
final_idx_test = pd.date_range(start='2016-01-31', end='2021-12-31', freq='7d')
final_df_calib = pd.DataFrame(index=final_idx_calib)
final_df_test = pd.DataFrame(index=final_idx_test)
final_df_test.shape

DatetimeIndex(['2001-02-06', '2001-02-13', '2001-02-20', '2001-02-27',
               '2001-03-06', '2001-03-13', '2001-03-20', '2001-03-27',
               '2001-04-03', '2001-04-10',
               ...
               '2015-10-27', '2015-11-03', '2015-11-10', '2015-11-17',
               '2015-11-24', '2015-12-01', '2015-12-08', '2015-12-15',
               '2015-12-22', '2015-12-29'],
              dtype='datetime64[ns]', length=778, freq='7D')


(309, 0)

In [27]:
# Y_pred und T_ival können schon eingelesen werden in den dataframe von begin
print(Y_pred_calib.shape, final_df_calib.shape)
final_df_calib['head forecast'] = Y_pred_calib
final_df_calib['95% lower'] = T_ival_calib[0]
final_df_calib['95% upper'] = T_ival_calib[1]
final_df_calib

(778, 1) (778, 0)


,head forecast,95% lower,95% upper
2001-02-06,347.049447,345.120556,348.978337
2001-02-13,346.754748,345.129894,348.379602
2001-02-20,346.969714,345.456676,348.482753
2001-02-27,346.634675,344.904849,348.364502
2001-03-06,346.535094,344.517587,348.552602
...,...,...,...
2015-12-01,347.622077,346.894018,348.350137
2015-12-08,347.625582,346.750824,348.500339
2015-12-15,347.723306,346.685345,348.761266
2015-12-22,347.907027,346.757604,349.056449


In [28]:
final_df_test['head forecast'] = yhat_mean
final_df_test['95% lower'] = T_ival_utf[0]
final_df_test['95% upper'] = T_ival_utf[1]
final_df_test

,head forecast,95% lower,95% upper
2016-01-31,346.688326,345.169268,348.207385
2016-02-07,347.020415,345.748828,348.292002
2016-02-14,347.527487,346.551283,348.503691
2016-02-21,347.501353,346.442650,348.560057
2016-02-28,347.562112,346.523569,348.600656
...,...,...,...
2021-11-28,347.593001,345.933375,349.252626
2021-12-05,347.058405,345.038989,349.077821
2021-12-12,346.697711,344.567641,348.827782
2021-12-19,346.578275,344.307195,348.849354


In [29]:
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/submissions'
chdir(csv_file_path)
final_df_calib.to_csv('Sweden2_calib.csv', float_format='%.2f')
final_df_test.to_csv('Sweden2_test.csv', float_format='%.2f')